In [2]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
ping_data = pd.read_csv('./data/ping_data.csv')
ping_data_wireless = pd.read_csv('./data/ping_data_rp59adf374.csv')
ping_data_ethernet = pd.read_csv('./data/ping_data_rp829c7e0e.csv')

In [4]:
print(ping_data_ethernet.shape)
print(ping_data_wireless.shape)
print(ping_data.shape)

(69745, 7)
(72463, 7)
(142208, 7)


In [5]:
ping_data.head()

,timestamp,location,source,local_dest,local_avg,remote_dest,remote_avg
0,1.559041e+09,lower-level,wireless,192.168.0.1,4.39,142.254.217.53,9.09
1,1.559041e+09,lower-level,wireless,192.168.0.1,0.49,142.254.217.53,16.75
2,1.559041e+09,lower-level,wireless,192.168.0.1,0.56,142.254.217.53,8.31
3,1.559041e+09,lower-level,wireless,192.168.0.1,1.64,142.254.217.53,9.40
4,1.559041e+09,lower-level,wireless,192.168.0.1,1.18,142.254.217.53,9.07


In [6]:
ping_data.groupby('source').head()

,timestamp,location,source,local_dest,local_avg,remote_dest,remote_avg
0,1.559041e+09,lower-level,wireless,192.168.0.1,4.39,142.254.217.53,9.09
1,1.559041e+09,lower-level,wireless,192.168.0.1,0.49,142.254.217.53,16.75
2,1.559041e+09,lower-level,wireless,192.168.0.1,0.56,142.254.217.53,8.31
3,1.559041e+09,lower-level,wireless,192.168.0.1,1.64,142.254.217.53,9.40
4,1.559041e+09,lower-level,wireless,192.168.0.1,1.18,142.254.217.53,9.07
72463,1.559041e+09,lower-level,ethernet,192.168.0.1,0.19,142.254.217.53,8.14
72464,1.559041e+09,lower-level,ethernet,192.168.0.1,0.15,142.254.217.53,7.75
72465,1.559041e+09,lower-level,ethernet,192.168.0.1,0.15,142.254.217.53,8.68
72466,1.559041e+09,lower-level,ethernet,192.168.0.1,0.15,142.254.217.53,8.08
72467,1.559041e+09,lower-level,ethernet,192.168.0.1,0.15,142.254.217.53,8.20


In [17]:
ping_data['date_time'] = ping_data['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%d/%m/%Y %H:%M:%S'))
ping_data_ethernet['date_time'] = ping_data_ethernet['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%d/%m/%Y %H:%M:%S'))
ping_data.head()

,timestamp,location,source,local_dest,local_avg,remote_dest,remote_avg,date_time
0,1.559041e+09,lower-level,wireless,192.168.0.1,4.39,142.254.217.53,9.09,28/05/2019 12:55:09
1,1.559041e+09,lower-level,wireless,192.168.0.1,0.49,142.254.217.53,16.75,28/05/2019 12:55:19
2,1.559041e+09,lower-level,wireless,192.168.0.1,0.56,142.254.217.53,8.31,28/05/2019 12:55:29
3,1.559041e+09,lower-level,wireless,192.168.0.1,1.64,142.254.217.53,9.40,28/05/2019 12:55:39
4,1.559041e+09,lower-level,wireless,192.168.0.1,1.18,142.254.217.53,9.07,28/05/2019 12:55:49


In [18]:
datetime.datetime.strptime(ping_data['date_time'][1], '%d/%m/%Y %H:%M:%S') - datetime.datetime.strptime(ping_data['date_time'][0], '%d/%m/%Y %H:%M:%S')

datetime.timedelta(seconds=10)

In [19]:
datetime.datetime.strptime(ping_data_ethernet['date_time'][len(ping_data_ethernet) - 1], '%d/%m/%Y %H:%M:%S') - datetime.datetime.strptime(ping_data_ethernet['date_time'][0], '%d/%m/%Y %H:%M:%S')

datetime.timedelta(days=7, seconds=45609)